<a href="https://colab.research.google.com/github/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/projeto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Simplificação de Texto

Nas primeiras abordagens, houve uma tentativa de produzir uma rede neural que tivesse como entrada o *token frequence* e saida outro *token frequence*.
 Apesar disso poder ser considerado um resultado válido para a tentativa de melhorar a classificação de texto, não podemos considerar uma frase como um conjunto de palavras com a propriedade da comutatividade.
  A ordem das palavras acaba importando muito para os humanos.

## importando bugingangas

In [0]:
import numpy as np
import csv
import random
#import pandas as pd
from urllib import request as req
from gensim.models import keyedvectors as kv

## Paper With Code

~~Após um gole de sorte, eu acabo por encontrar essa maravilha~~ chamado de [Paper With Code](https://paperswithcode.com/sota/document-summarization-on-cnn-daily-mail), é um ~~bom~~ compilado de resultados cientificos sobre determinados problemas da computação. O primeiro artigo do link de cima, mostra uma solução com ROGUE-1 de 43.83 para o problema de sumarização de documentos como o *state of the art*.

![paper with code](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/paperwithcode.png?raw=true)
<center>paper with code</center>

No artigo *Text Summarization with Pretrained Encoders* (LIU e LAPATA, 2019), os autores representam o texto como *Bert*, que é gerado por uma rede neural e é um caso especial do *word2vec*.




## Representação do Bert word2vec

*Kyubyong* criou um preset do Bert, um dicionário onde cada palavra é representada por um vetor de 768 valores decimais.
Visto algumas limitações das plataformas *Google Colab* e do *Github*, serão usadas apenas 300 instâncias do CNN, que gerou um dicionário reduzido de apenas 25mb.

![Bert is Evil](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/audio-banner.jpg?raw=true)
<center>Bert is Evil</center>

## Base de dados da Cable News Network

A base de dados pode ser encontrado [nesse github da google](https://github.com/google-research-datasets/sentence-compression), no formato *Json*. Essa base de dados é formada com notícias da CNN com a primeira linha da noticia, com o título, com todos os bigramas possíveis e com informações do TAG. Porém no trabalho apenas será usado a primeira linha como entrada e o título como saída.

![Json](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/Jasonf.jpg?raw=true)
<center>Json</center>

## Mesclar o bert com o json.

Basicamente esse script carrega o vocabulário do dicionário bert, onde a partir da segunda linha, cada linha é uma palavra, seguida de sua representação vetorial com 768 valores.
Eventualmente aparecem algumas palavras estranhas como "##,".
Essa palavra significa somente que a palavra anterior termina com uma virgula.

Depois de carregar o vocabulário, ele começa a ler as noticias do json.
Ele segmenta uma noticia, a transformando em um dicionário do python, procura os textos alvos
### jfk['graph']['sentence'] e jfk['headline'].

Com os textos em mãos, ele cata as palavras de cada texto, criando uma matriz de tamanho 768*2 pela quantidade de linhas do subtitulo da noticia.

Observar que para esse código rodar ele deve estar na seguinte pasta ~~por isso está comentado~~. Com o json nomeado *in.json*, com o bert nomeado *dicionariolongo.vec* e uma pasta vazia chamada de *instancias*. O script vai gerar um *dicionario.csv* e varias noticias dentro da pasta instancias. Infelizmente algumas noticias tem caracters que não permitem seu uso como nome do arquivo, por isso todas as noticias foram nomeadas de *arquivo_n.csv* ~~Outro motivo para você guardar seus dados em um único arquivo json~~.

>O diretório do arquivo.
>>in.json \n
dicionariolongo.vec \n
preprobert.py
>>> instancias


In [133]:
"""
#texto para matriz Bert

#!/usr/bin/python
# -*- coding: UTF-8 -*-

import pandas as pd
import csv

import nltk
import numpy as np

import json
import fileinput

import codecs

import os

dici = {}
fins = {}
dico = {} #dicionario reduzido


#carrega o dicionario bert
with open("dicionariolongo.vec", "r", encoding="utf-8") as d:
    for i in d:
        if len(i) > 200: #isso aqui eh para tirar a primeira linha, altura x largura
            try:
                a = i.split()
                if '##' in a[0]:
                    fins[a[0][2:].lower()] = []
                    for j in a[1:]:
                        fins[a[0][2:].lower()].append(float(j))
                elif (not a[0].lower() in dici) or a[0].islower():
                    dici[a[0].lower()] = []
                    for j in a[1:]:
                        dici[a[0].lower()].append(float(j))
            except ValueError:
                print("problema com uma palavra")



with open("in.json", 'rb') as f:
    data = f.read()

data = str(data)[2:-1]
data = data.replace('\\n}', '\\n}-----')
data = data.split('-----')

blanckl = []
for i in range(768):
    blanckl.append('')

instancias = 0
for p in data[0:-1]:
    entrada = []
    saida = []
    try:
        jfk = json.loads(codecs.decode(p, 'unicode_escape'))
        arquivo = jfk['graph']['sentence'].lower().replace("/","").replace("\\","")
        lg = jfk['graph']['sentence'].lower().split()
        st = jfk['headline'].lower().split()

        for i in lg: #isso deveria ser uma funcao, addlist() #semTempo
            if i in dico: #dicionario reduzido
                entrada.append(dico[i])
            elif i in dici:
                dico[i] = dici[i]
                entrada.append(dico[i])
            else:
                for j, jj in fins.items():
                    if j in i[-len(j):]:
                        i = i[:-len(j)]
                        if i in dico: #assim ela poderia ser chamada aqui
                            entrada.append(dico[i])
                        elif i in dici:
                            dico[i] = dici[i]
                            entrada.append(dico[i])
                        if j in dico:
                            entrada.append(dico[j])
                        else:
                            dico[j] = fins[j]
                            entrada.append(dico[j])
                        break
        for i in st: #isso deveria ser uma funcao, addlist() #semTempo
            if i in dico: #dicionario reduzido
                saida.append(dico[i])
            elif i in dici:
                dico[i] = dici[i]
                saida.append(dico[i])
            else:
                for j, jj in fins.items():
                    if j in i[-len(j):]:
                        i = i[:-len(j)]
                        if i in dico: #assim ela poderia ser chamada aqui
                            saida.append(dico[i])
                        elif i in dici:
                            dico[i] = dici[i]
                            saida.append(dico[i])
                        if j in dico:
                            saida.append(dico[j])
                        else:
                            dico[j] = fins[j]
                            saida.append(dico[j])
                        break
    except ValueError:
        print("Houve um Erro")

        
    sv = os.getcwd() + '\\instancias\\arquivo_%i'%instancias + '.csv'
    instancias += 1
    try:
        with open(sv, 'w', newline = '') as file:
            writer = csv.writer(file)
            tam = 0
            while tam < len(entrada):
                tm = []
                #print(type(tm))
                tm.extend(entrada[tam])
                if tam < len(saida):
                    tm.extend(saida[tam])
                else:
                    tm.extend(blanckl)
                tam += 1
                writer.writerow(tm)
    except ValueError:
        print("erro no json")

with open("dicionario.csv", "w", newline = '') as file:
    writer = csv.writer(file)
    for i, ii in dico.items():
	tm = []
	tm.append(i)
	tm.extend(ii)
        writer.writerow(tm)
"""

'\n#texto para matriz Bert\n\n#!/usr/bin/python\n# -*- coding: UTF-8 -*-\n\nimport pandas as pd\nimport csv\n\nimport nltk\nimport numpy as np\n\nimport json\nimport fileinput\n\nimport codecs\n\nimport os\n\ndici = {}\nfins = {}\ndico = {} #dicionario reduzido\n\n\n#carrega o dicionario bert\nwith open("dicionariolongo.vec", "r", encoding="utf-8") as d:\n    for i in d:\n        if len(i) > 200: #isso aqui eh para tirar a primeira linha, altura x largura\n            try:\n                a = i.split()\n                if \'##\' in a[0]:\n                    fins[a[0][2:].lower()] = []\n                    for j in a[1:]:\n                        fins[a[0][2:].lower()].append(float(j))\n                elif (not a[0].lower() in dici) or a[0].islower():\n                    dici[a[0].lower()] = []\n                    for j in a[1:]:\n                        dici[a[0].lower()].append(float(j))\n            except ValueError:\n                print("problema com uma palavra")\n\n\n\nwit

## Abrindo uma noticia (bert) e convertendo para texto.

In [134]:
# Abrindo o dicionario criado com o outro script
dicurl = 'https://raw.githubusercontent.com/x-channel/Mining-Text-Simplifica-o-de-Texto/master/dataset/dicionario.csv'
#dicloader = req.urlopen(dicurl)
vetores = []
palavras = []

with req.urlopen(dicurl) as f:
  meucsv = f.read().decode('charmap')
  meucsv = meucsv.split('\n')[:-1]
  for i in meucsv:
    j = i.replace('\r','').split(',')
    palavras.append(j[0])
    vetores.append(j[1:])

# colocando o dicionario no gensim.
dicio = kv.Word2VecKeyedVectors(768)
dicio.add(palavras, np.array(vetores).astype(float))

#busca por um vetor parecido
five = dicio.get_vector('five')
friends = dicio.get_vector('friends')

print(dicio.similar_by_vector(five, 1), dicio.similar_by_vector(friends, 1))

#abrindo a noticia
#nn = input("digite um numero entre 0 e 199: ")
nn = 1
noticia = 'https://raw.githubusercontent.com/x-channel/Mining-Text-Simplifica-o-de-Texto/master/dataset/noticias/arquivo_%i.csv'%nn

pastaNoticias = 'https://raw.githubusercontent.com/x-channel/Mining-Text-Simplifica-o-de-Texto/master/dataset/noticias/%s_%i.csv'

def abrirNoticia(urlnoticia):
  sentence = []
  head = []
  with req.urlopen(urlnoticia) as f:
    meucsv = f.read().decode('charmap')
    meucsv = meucsv.split('\n')[:-1]
    for i in meucsv:
      j = i.replace('\r',',').replace(',,', ',0.0,').replace(',,', ',0.0,')[:-1]
      j = j.split(',')
      sentence.append(j[:768])
      head.append(j[768:])
  sentence = np.array(sentence).astype(float)
  head = np.array(head).astype(float)
  return sentence, head

def abrirNoticias(urlfolder, total, nome = 'arquivo'):
  primeiro = []
  cabecalh = []
  for i in range(total):
    par = abrirNoticia(urlfolder%(nome,i))
    primeiro.append(par[0])
    cabecalh.append(par[1])
  return np.array(primeiro),np.array(cabecalh)


def vec2head(matriz, model, li):
  head = []
  for i in matriz:
    j = model.similar_by_vector(i,1)
    if j[0][1] > li:
      head.append(j)
  return head

s, h = abrirNoticia(noticia)

titulo = vec2head(h, dicio, 0.5)
print (titulo)

noticias,cabecalhos = abrirNoticias(pastaNoticias, 200)
print(vec2head(noticias[1], dicio, 0.5))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('five', 1.0)] [('friends', 1.0)]
[[('several', 1.0)], [('school', 1.0)], [('districts', 1.0)], [('hold', 1.0)], [('classes', 1.0)], [('on', 1.0)], [('president', 0.6351367831230164)], [("##'", 0.9999998807907104)], [('day', 1.0000001192092896)], [('to', 1.0)], [('make', 1.0)], [('up', 1.0000001192092896)], [('for', 1.0)], [('days', 1.0)], [('missed', 1.0)]]
[[('several', 1.0)], [('school', 1.0)], [('districts', 1.0)], [('in', 1.0)], [('hampton', 1.0)], [('roads', 1.0)], [('are', 0.9999999403953552)], [('holding', 1.0)], [('classes', 1.0)], [('this', 1.0000001192092896)], [('president', 0.6351367831230164)], [("##'", 0.9999998807907104)], [('day', 1.0000001192092896)], [('to', 1.0)], [('make', 1.0)], [('up', 1.0000001192092896)], [('for', 1.0)], [('days', 1.0)], [('missed', 1.0)], [('because', 1.0)], [('of', 1.0)], [('the', 1.0)], [("##'", 0.696808934211731)]]


## Dividindo a base de dados

Aqui a base de dados é dividida em: Treinamento, validação e teste.

OBS: não encontrei isso implementado nem no scikit learn.

In [135]:
#pading, para tudo ter o mesmo tamanho

maxlen = 0

for i in noticias:
  if i.shape[0] > maxlen:
    maxlen = i.shape[0]

for i in range(len(noticias)):
  pad = np.zeros((maxlen, 768))
  pad[:noticias[i].shape[0],:] = noticias[i]
  noticias[i] = pad
  pad = np.zeros((maxlen, 768))
  pad[:cabecalhos[i].shape[0],:] = cabecalhos[i]
  cabecalhos[i] = pad

print(maxlen)

56


In [136]:
treinamento = []
#validacao = []
teste = []

xt = []
yt = []

xr = []
yr = []

atreino = 0.9
#avalidacao = 0.3

d = []
for i in range(len(noticias)):
  d.append(i)

random.shuffle(d)

# deixar essa linha para validar os parametros dos primeiros testes
d = [117, 135, 181, 2, 129, 167, 65, 183, 107, 104, 158, 111, 69, 194, 8, 101, 21, 35, 31, 188, 106, 196, 148, 198, 67, 60, 102, 82, 16, 88, 119, 61, 11, 115, 113, 56, 169, 98, 64, 40, 49, 162, 36, 127, 157, 66, 164, 180, 41, 138, 62, 34, 72, 178, 27, 189, 121, 154, 96, 14, 133, 145, 97, 43, 199, 51, 25, 163, 155, 47, 70, 150, 12, 30, 123, 195, 32, 55, 18, 176, 171, 68, 175, 120, 110, 59, 141, 6, 23, 44, 103, 151, 125, 130, 79, 73, 173, 1, 58, 165, 118, 46, 39, 191, 10, 74, 166, 24, 147, 131, 190, 20, 156, 26, 22, 187, 182, 75, 63, 52, 9, 132, 87, 5, 144, 192, 42, 142, 90, 85, 143, 13, 153, 174, 122, 139, 184, 128, 19, 50, 161, 172, 168, 83, 48, 71, 185, 53, 126, 4, 29, 86, 15, 7, 92, 45, 197, 76, 134, 37, 54, 152, 57, 84, 112, 3, 28, 93, 0, 109, 136, 177, 77, 170, 100, 146, 137, 179, 80, 33, 17, 124, 89, 193, 38, 160, 78, 95, 140, 114, 159, 81, 186, 99, 108, 105, 116, 94, 149, 91]

for i in range(len(noticias)):
  if i < atreino*len(noticias):
    xt.append(noticias[d[i]])
    yt.append(cabecalhos[d[i]])
  else:
    xr.append(noticias[d[i]])
    yr.append(cabecalhos[d[i]])




xt = np.array(xt)
yt = np.array(yt)

xr = np.array(xr)
yr = np.array(yr)

xt = xt.reshape(xt.shape[0], maxlen, 768, 1)
yt = yt.reshape(yt.shape[0], maxlen, 768, 1)

xr = xr.reshape(xr.shape[0], maxlen, 768, 1)
yr = yr.reshape(yr.shape[0], maxlen, 768, 1)

print(len(xt), len(yt))
print(len(xr), len(xr[1]), len(xr[1][1]))

'''
for i in range(len(noticias)):
  if i < atreino*len(noticias):
    treinamento.append(noticias[d[i]])
  elif i < (atreino+avaliacao)*len(noticias):
    validacao.append(noticias[d[i]])
  else:
    teste.append(noticias[d[i]])'''


180 180
20 56 768


'\nfor i in range(len(noticias)):\n  if i < atreino*len(noticias):\n    treinamento.append(noticias[d[i]])\n  elif i < (atreino+avaliacao)*len(noticias):\n    validacao.append(noticias[d[i]])\n  else:\n    teste.append(noticias[d[i]])'

## Construindo a Rede Neural

In [0]:
#Importando o keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Deconvolution2D, Reshape, Dropout
import keras

In [138]:
model = Sequential()
#model.add(Conv2D(2, kernel_size=(3,3), activation = "tanh", input_shape=(maxlen,768,1)))
model.add(Conv2D(2, kernel_size=(3,3), activation = "tanh", input_shape=(maxlen,768,1),padding="same"))
model.add(Conv2D(4, (2,2), activation = "tanh",padding="same"))
print(model.input_shape)
print(model.output_shape)
#model.add(MaxPooling2D(pool_size = (2,2)))
#print(model.output_shape)

model.add(Conv2D(8, (2,2), activation = "tanh",padding="same"))
model.add(Dropout(0.8))

model.add(Conv2D(16, (2,2), activation = "tanh",padding="same"))
model.add(Dropout(0.8))

model.add(Conv2D(32, (2,2), activation = "tanh",padding="same"))
model.add(Dropout(0.8))

model.add(Conv2D(64, (2,2), activation = "tanh",padding="same"))
model.add(Dropout(0.8))

model.add(Conv2D(128, (2,2), activation = "tanh",padding="same"))
model.add(Dropout(0.8))

print()
model.add(Deconvolution2D(1, (2,2),padding="same", dilation_rate =2))
print(model.output_shape)
print(model.summary())

#model.add(Reshape((maxlen,768,1)))
#print(model.output_shape)

#model.add(Conv2DTranspose(1,(2,2),output_shape=(768,), activation="tanh"))
#print(model.output_shape)
#rs = Reshape((maxlen,768,1))
#print(rs.output_shape)
#print(rs.input_shape)


model.compile(optimizer='adam',loss='mse',metrics=["acc"])


(None, 56, 768, 1)
(None, 56, 768, 4)

(None, 56, 768, 1)
Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_75 (Conv2D)           (None, 56, 768, 2)        20        
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 56, 768, 4)        36        
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 56, 768, 8)        136       
_________________________________________________________________
dropout_20 (Dropout)         (None, 56, 768, 8)        0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 56, 768, 16)       528       
_________________________________________________________________
dropout_21 (Dropout)         (None, 56, 768, 16)       0         
_____________________________________________________________

## Treinando a rede

In [139]:
model.fit(xt,yt,epochs=100)

Epoch 1/100
180/180 [==============================] - 4s 20ms/step - loss: 0.0402 - acc: 0.8319
Epoch 2/100
180/180 [==============================] - 1s 8ms/step - loss: 0.0167 - acc: 0.8528
Epoch 3/100
180/180 [==============================] - 1s 8ms/step - loss: 0.0082 - acc: 0.8572
Epoch 4/100
180/180 [==============================] - 1s 8ms/step - loss: 0.0046 - acc: 0.8577
Epoch 5/100
180/180 [==============================] - 1s 8ms/step - loss: 0.0030 - acc: 0.8577
Epoch 6/100
180/180 [==============================] - 1s 8ms/step - loss: 0.0021 - acc: 0.8577
Epoch 7/100
180/180 [==============================] - 1s 8ms/step - loss: 0.0017 - acc: 0.8577
Epoch 8/100
180/180 [==============================] - 1s 8ms/step - loss: 0.0014 - acc: 0.8577
Epoch 9/100
180/180 [==============================] - 1s 8ms/step - loss: 0.0012 - acc: 0.8577
Epoch 10/100
180/180 [==============================] - 1s 8ms/step - loss: 0.0011 - acc: 0.8577
Epoch 11/100
180/180 [================

##Teste da rede

Aqui a rede é testada, sem produzir de fato saidas legíveis, mas apenas mostrando uma acurácia da rede em relação ao vetor bert.

In [140]:
print(xt.shape)

aaaaa = xr[0].reshape(1, 56, 768, 1)
bbbbb = yr[0].reshape(56,768)

ttttt = model.predict(aaaaa)

(180, 56, 768, 1)


In [141]:
ttttt = ttttt.reshape(56,768)
print(ttttt.shape)

asdf = vec2head(ttttt,dicio,0.5)
print(asdf)

fdsa = vec2head(bbbbb,dicio,0.5)
print(fdsa)

(56, 768)
[[('201', 0.6859637498855591)], [('201', 0.6856739521026611)], [('201', 0.688129186630249)], [('201', 0.6882786750793457)], [('201', 0.687862753868103)], [('201', 0.6877453923225403)], [('201', 0.6847168207168579)], [('201', 0.6844584941864014)], [('201', 0.684297502040863)], [('201', 0.6865218281745911)], [('201', 0.6867110729217529)], [('201', 0.6875910758972168)], [('201', 0.6872336864471436)], [('201', 0.6859800219535828)], [('201', 0.6853822469711304)], [('201', 0.6861602663993835)], [('201', 0.6865890026092529)], [('201', 0.6845738887786865)], [('201', 0.6838492155075073)], [('201', 0.6844849586486816)], [('201', 0.6845182180404663)], [('201', 0.6833680868148804)], [('201', 0.6863068342208862)], [('201', 0.6868082284927368)], [('201', 0.6873829364776611)], [('201', 0.6882473230361938)], [('201', 0.6862123012542725)], [('201', 0.6848485469818115)], [('201', 0.683991014957428)], [('201', 0.6828604936599731)], [('201', 0.6811395287513733)], [('201', 0.683478593826294)], [(

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## Teste de predição

Aqui a rede é rodada como teste. A metrica mais usada para esse teste é o ROUGE-1, que conta quantas palavras da saída do sistema está dentro do texto de referência, depois divide pelo número de palavras no texto de referência.

In [142]:


#rouge-1
def rouge(src, ref):
  acertos = 0
  for i in src:
    if i in ref:
      acertos += 1
  return acertos/float(len(ref))

sistem = ['five', 'plane']
refere = ['five', 'in', 'plane']

si = []
re = []

for i in asdf:
  si.append(i[0])

for i in fdsa:
  re.append(i[0])

print(rouge(sistem, refere))
print(rouge(si,re))



0.6666666666666666
0.0


# SHAME!

Certo, o resultado de 85% de precisão da rede neural não produziu nenhuma sentença compatível com a sua referência. Vamos tentar outra coisa.

## Classificação

Há uma base de dados que compreende recomendações dos jogos da steam. https://github.com/mulhod/steam_reviews


Porém aqui será usado somente o word2vec, visto que a base de dados é outra e não é possível somente importar o dicionário do bert.


In [0]:
# To reimportando para nao precisar rodar tudo.
import gensim
import numpy as np
import json
import keras

from urllib import request as req
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

## importando a base de dados.

In [144]:
#urlpastareviews = "https://github.com/mulhod/steam_reviews/tree/master/data"
vetorarqui = ["Arma_3.jsonlines","Counter_Strike.jsonlines","Counter_Strike_Global_Offensive.jsonlines","Dota_2.jsonlines","Football_Manager_2015.jsonlines","Garrys_Mod.jsonlines","Grand_Theft_Auto_V.jsonlines","Sid_Meiers_Civilization_5.jsonlines","Team_Fortress_2.jsonlines","The_Elder_Scrolls_V.jsonlines","Warframe.jsonlines"]
cadaReview = "https://raw.githubusercontent.com/mulhod/steam_reviews/master/data/%s"

#user/game, rating
gamerat = [[],[]]
reviews = []

erros = 0

#aqui o vetor reviews sera acrescido de instancias.
for i in vetorarqui:
  with req.urlopen(cadaReview%i) as f:
    data = f.read().decode('charmap')
  data = data.replace('}\n', '}-----')
  data = data.split('-----')
  print("reviews coletadas para esse jogo",len(data))
  for j in data:
    try:
      jfk = json.loads(j)
      # infelizmente a base de dados contem instancias duplicadas
      # Eventualmente um jogador comentou em dois jogos
      # Eventualmente os jogadores mudam de nome, entao eh mais seguro filtrar pelo "orig_url"
      #hum += 1
      user = "%s in %s"%(jfk["orig_url"], i[:-10])
      revi = jfk["review"]
      revi = gensim.utils.simple_preprocess(revi)
      rati = jfk["rating"]
      if (not (user in gamerat[0])):
        if len(revi) > 5 and len(revi) < 129:
          gamerat[0].append(user)
          reviews.append(revi)
          gamerat[1].append(rati)
        elif len(revi) > 128:
          gamerat[0].append(user)
          reviews.append(revi[:128])
          gamerat[1].append(rati)
    except:
      erros += 1
  print(reviews[-1])
  print ("Ha %i instancias\nUm total de %i instancias ignoradas"%(len(reviews),erros))

reviews coletadas para esse jogo 7274
['simply', 'amazing', 'fun', 'to', 'play', 'insurgency', 'and', 'king', 'of', 'the', 'hill']
Ha 881 instancias
Um total de 135 instancias ignoradas
reviews coletadas para esse jogo 6234
['just', 'an', 'absolute', 'brilliant', 'game', 'you', 'can', 'play', 'on', 'any', 'server', 'you', 'want', 'and', 'you', 'will', 'have', 'lot', 'of', 'fun']
Ha 2083 instancias
Um total de 336 instancias ignoradas
reviews coletadas para esse jogo 7412
['you', 'shoot', 'chickens', 'in', 'this', 'game']
Ha 3001 instancias
Um total de 691 instancias ignoradas
reviews coletadas para esse jogo 9792
['yeah', 'best', 'game', 'cant', 'imagine', 'if', 'this', 'game', 'if', 'rlly', 'closed']
Ha 4550 instancias
Um total de 774 instancias ignoradas
reviews coletadas para esse jogo 1532
['well', 'there', 'was', 'looking', 'to', 'enjoy', 'the', 'nexr', 'edition', 'off', 'fm', 'oh', 'dear', 'who', 'on', 'earth', 'made', 'this', 'one', 'must', 'be', 'someone', 'that', 'has', 'never

## Criando o Word2Vec

In [145]:
dicionario = gensim.models.Word2Vec(
        reviews,
        size=128,
        window=10,
        min_count=2,
        workers=100)
dicionario.train(reviews, total_examples=len(reviews), epochs=20)

(5924266, 8172540)

In [146]:
print(len(dicionario["good"]))

print(len(dicionario.wv.vocab))

print(dicionario["good"])


128
8669
[-1.2259949e+00  1.9993063e+00  5.8566559e-02  2.2102705e-01
  1.4508879e+00  1.5133581e+00 -1.2312982e+00  1.1861554e+00
 -3.6624763e+00 -1.2328969e+00  1.7400835e+00 -2.0218928e-01
  7.5327468e-01  4.5907187e-01  2.4640264e-01  2.4597011e+00
  6.4571917e-01  8.0510199e-01 -9.0763412e-02  1.1370085e+00
  7.6558316e-01  1.8016051e+00  1.0349616e+00 -1.1807398e+00
 -1.0684725e+00 -1.2889031e+00 -7.5821739e-01  9.0404582e-01
  2.9936960e-01 -1.5309575e+00 -1.6541843e+00  4.3744531e-02
  1.0518887e+00 -2.5894451e-01 -9.8706551e-02  2.1057298e+00
 -1.5026595e-01  1.4706271e+00  4.7380897e-01 -1.9008769e+00
 -1.1936704e+00  1.8315812e+00 -2.7377719e-01  1.1052895e+00
 -7.2269541e-01 -1.0157968e+00  8.1258541e-01 -6.4162338e-01
  1.3447130e-01  1.7760757e+00  2.5003834e+00 -2.0340405e+00
  1.7368201e+00  1.1100343e+00 -9.6069515e-01 -2.8983623e-01
 -2.7555761e-01  2.5051558e-01 -4.9124801e-01 -3.9798239e-01
  4.4681627e-01 -1.3490374e+00  1.0653763e+00  5.4898262e-01
 -3.2799783e-01

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


## Filtrando as instancias

Devido algumas pequenas dificuldades, o projeto sera simplificado com uma filtragem das instancias. Com uma base de dados equilibrada, 50% de instancias recomendadas e 50% de instancias não recomendadas, a estratégia de dizer que todas as entradas são recomendadas não funcionará.

In [147]:
print(len(reviews))
print(len(gamerat))
print(len(gamerat[0]))

reviewsold = reviews
gameratold = gamerat

reviews = []
gamerat = [[],[]]

print(len(gameratold))
print(len(gameratold[1]))
len(reviewsold)

rm = 0
nr = 0

for i in gameratold[1]:
  if i == 'Recommended':
    rm += 1
  else:
    nr += 1

print(rm, nr)

rm = min(rm, nr)
nr = rm

for i in range(len(reviewsold)):
  if rm > 0 and gameratold[1][i] == 'Recommended':
    reviews.append(reviewsold[i])
    gamerat[0].append(gameratold[0][i])
    gamerat[1].append(gameratold[1][i])
    rm -= 1
  if rn > 0 and gameratold[1][i] == 'Not Recommended':
    reviews.append(reviewsold[i])
    gamerat[0].append(gameratold[0][i])
    gamerat[1].append(gameratold[1][i])
    nr -= 1

print(len(reviews), len(gamerat[1]))

10548
2
10548
2
10548
9072 1476
2952 2952


## Transformando cada review em uma matriz

Como usado anteriormente, mas sem o bert, cada review sera transformada em uma matriz, onde cada linha é uma palavra com 128 valores.

In [148]:
reviewsmat = []

for i in reviews:
  mm = []
  for j in i:
    try:
      mm.append(dicionario[j])
    except:
      #print("Faltou")
      pass
  mm = np.array(mm)
  reviewsmat.append(mm)



print(len(reviews))
print(len(reviewsmat))
print(len(reviewsmat[0]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


2952
2952
128


In [149]:
#pading, para tudo ter o mesmo tamanho

maxlen = 0

revFiltradas = []

for i in reviewsmat:
  if i.shape[0] > maxlen:
    maxlen = i.shape[0]

erros = 0

for i in range(len(reviewsmat)):
  try:
    pad = np.zeros((maxlen, 128))
    pad[:reviewsmat[i].shape[0],:] = reviewsmat[i]
    revFiltradas.append(pad)
  except:
    erros += 1

print(maxlen, erros)

128 0


In [150]:
#Reshaping

inst = len(revFiltradas)

x = np.array(revFiltradas)

x = x.reshape(inst, maxlen, 128, 1)

len(x)

2952

## One hot encoder

In [151]:
#Outputs

ardata = np.array(gamerat[1])
print(ardata)


# https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(ardata)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[0, :])])
print(inverted)


y = keras.utils.to_categorical(onehot_encoded, 2)

print(y)

['Recommended' 'Recommended' 'Recommended' ... 'Not Recommended'
 'Not Recommended' 'Not Recommended']
[1 1 1 ... 0 0 0]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
['Recommended']
[[[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 ...

 [[0. 1.]
  [1. 0.]]

 [[0. 1.]
  [1. 0.]]

 [[0. 1.]
  [1. 0.]]]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Divisão treino/teste

In [152]:
## divisao

print (inst)

x_tr = x[:2361] #80% da base de dados
x_te = x[2361:] #20% da base de dados

y_tr = onehot_encoded[:2361]
y_te = onehot_encoded[2361:]

2952


## Classificador

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout

In [0]:
#### Recall 
#### https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

In [155]:
cls = Sequential()
cls.add(Conv2D(4, kernel_size=(3,3), activation = "tanh", input_shape=(maxlen,128,1),padding="same"))
cls.add(MaxPooling2D(padding='same'))
cls.add(Conv2D(8, kernel_size=(3,3), activation = "tanh", input_shape=(maxlen,128,1),padding="same"))
cls.add(Dropout(0.8))
cls.add(MaxPooling2D(padding='same'))
cls.add(Conv2D(16, kernel_size=(3,3), activation = "tanh", input_shape=(maxlen,128,1),padding="same"))
cls.add(MaxPooling2D(padding='same'))
cls.add(Conv2D(32, kernel_size=(3,3), activation = "tanh", input_shape=(maxlen,128,1),padding="same"))
cls.add(MaxPooling2D(padding='same'))
cls.add(Conv2D(64, kernel_size=(3,3), activation = "tanh", input_shape=(maxlen,128,1),padding="same"))
cls.add(Dropout(0.8))
cls.add(MaxPooling2D(padding='same'))
cls.add(Flatten())
cls.add(Dense(8, activation = "tanh"))
cls.add(Dropout(0.8))
cls.add(Dense(2, activation = "softmax"))

print(cls.summary())
print(cls.input_shape)
print(cls.output_shape)


cls.compile("adam", "mse", ["acc", recall_m])

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_82 (Conv2D)           (None, 128, 128, 4)       40        
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 64, 64, 4)         0         
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 64, 64, 8)         296       
_________________________________________________________________
dropout_25 (Dropout)         (None, 64, 64, 8)         0         
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 32, 32, 8)         0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 32, 32, 16)        1168      
_________________________________________________________________
max_pooling2d_56 (MaxPooling (None, 16, 16, 16)      

In [156]:
cls.fit(x_tr, y_tr, batch_size=160, epochs = 300, validation_split=0.1)

Train on 2124 samples, validate on 237 samples
Epoch 1/300
2124/2124 [==============================] - 2s 986us/step - loss: 0.2680 - acc: 0.6577 - recall_m: 0.6558 - val_loss: 0.3301 - val_acc: 0.3840 - val_recall_m: 0.3840
Epoch 2/300
2124/2124 [==============================] - 0s 218us/step - loss: 0.2748 - acc: 0.6648 - recall_m: 0.6648 - val_loss: 0.3401 - val_acc: 0.3671 - val_recall_m: 0.3671
Epoch 3/300
2124/2124 [==============================] - 0s 216us/step - loss: 0.2674 - acc: 0.6676 - recall_m: 0.6676 - val_loss: 0.4202 - val_acc: 0.3038 - val_recall_m: 0.3038
Epoch 4/300
2124/2124 [==============================] - 0s 213us/step - loss: 0.2573 - acc: 0.6827 - recall_m: 0.6827 - val_loss: 0.4118 - val_acc: 0.2954 - val_recall_m: 0.2954
Epoch 5/300
2124/2124 [==============================] - 0s 212us/step - loss: 0.2466 - acc: 0.6822 - recall_m: 0.6822 - val_loss: 0.3539 - val_acc: 0.2996 - val_recall_m: 0.2996
Epoch 6/300
2124/2124 [==============================] - 0

In [157]:
cls.evaluate(x_te, y_te)

591/591 [==============================] - 0s 141us/step


[0.2296112567035075, 0.6785109984592335, 0.6785109984592335]

## Os erros metodológicos

1. Os parâmetros não tem referências, foram feitos no teste.

2. O Word2Vec computa todos os valores do banco de dados, o que pode enviezar os valores do dicionário. Como Rosa Weber "Pelo principio do colegiado, darei meu voto para seguir a maioria que será composta por mim".

3. Não há outro classificador para comparar.

4. Não há medida estatistica para saber qual a chance desse modelo realmente funcionar em problemas do mundo real.

## Referencias

